In [1]:
from datasets import load_dataset, Dataset
from torch.nn import LSTM


/Users/brianyao/anaconda3/envs/cse447/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("uonlp/CulturaX", "en", split='train', streaming=True)

In [7]:
ds_head = ds.take(5000)
first_example = None
for d in ds_head:
    print(d)
    first_example = d
    break

{'text': 'DOT Announces 2008 Exploration Program - Redorbit\nCALGARY, ALBERTA--(Marketwire - July 31, 2008) - DOT Resources Ltd. (TSX VENTURE:DOT) ("DOT" or the "Corporation") is pleased to announce it will be completing a deep 3D Induced Polarization/ Resistivity ("IP") survey on its Dot porphyry copper property (the "Property") located 17 kilometres south of the Highland Valley Mining complex, in central British Columbia. The purpose of the 3D IP survey is to test the depth extension of the copper-molybdenum mineralization outlined to date on the Southeast zone and to explore the area of low magnetic susceptibility for zones of sulphide mineralization.\nThe total field magnetometer survey completed over the Property in 2007 located a large area measuring 3 kilometres ("km") by 1 km of low magnetic susceptibility (see Figure 1) which is interpreted to reflect either:\n- alteration, or\n- the intersection of cross-cutting structures, or\n- a combination of both features\nThe porphyry d

In [18]:
print(ds_head)

IterableDataset({
    features: ['text', 'timestamp', 'url', 'source'],
    n_shards: 3072
})


In [6]:
def create_pairs(example):
    text = example["text"]
    pairs = []
    for i in range(1, len(text)):
        pairs.append({"input": text[:i], "output": text[i]})
    return pairs
print(type(d))
print(len(create_pairs(d)))
# transformed = Dataset.from_dict(d).map(create_pairs, remove_columns=["text", "timestamp"], batched=True)
# for t in transformed:
#     print(t)

<class 'dict'>
3242


In [10]:
chars = set()
num_chars = 0
for d in ds_head:
    num_chars += len(d['text'])
    # print(set(list(d['text'])))
    chars.update(d['text'])
print(len(chars))
print(num_chars)

1799
35475071
